### CSCI 544 HW2

In [1]:
import json
from collections import defaultdict

In [2]:
with open('./data/train.json') as f:
    train_data = json.load(f)

### Task 1: Vocabulary Creation (20 points)
Ran through the dataset and filter out words that have amount below threshold (make them \<unk\>)

In [3]:
# Task 1: Vocabulary Creation
dic = defaultdict(int)
threshold = 2
for i in range(len(train_data)):
    cur_sentence = train_data[i]['sentence']
    for w in cur_sentence:
        dic[w] += 1
sorted_list = sorted(dic.items(), key = lambda x:x[1], reverse = True)
unknown_count = 0
replace_count = 0
for k, v in sorted_list:
    if v < threshold:
        replace_count += 1
        unknown_count += v
vocab = {'<unk>': (0, unknown_count)}
for i, cur in enumerate(sorted_list):
    cur_w = cur[0]
    cur_v = cur[1]
    if cur_v < threshold: continue
    vocab[cur_w] =  (i + 1, cur_v)
    
print("Vocabulary size: ", len(vocab))
print("Replace amount: ", replace_count)


with open('vocab.txt', 'w') as f_out:
    for word, item in vocab.items():
        index = item[0]
        count = item[1]
        f_out.write(str(word)+'\t'+str(index)+'\t'+str(count)+'\n')

Vocabulary size:  23183
Replace amount:  20011


Q: What threshold value did you choose for identifying unknown words for replacement? </br>
Ans: 2 </br>
Q: What is the overall size of your vocabulary, and how many times does the special token ”< unk >” occur following the replacement process? </br>
Ans: Vocabulary overall size -> 23183, replacement occur -> 20011 

### Task 2: Model Learning (20 points)
Calculate the transition and emission probs for later implement

In [4]:
#Task 2: Model Learning
transition_data_count = defaultdict(int)
state_count = defaultdict(int)
emission_data_count = defaultdict(int)
sentence_count = len(train_data)
tag_list = []

for row in train_data:
    cur_s = row['sentence']
    cur_labels = row['labels']
    pre = '<s>'
    end_index = len(cur_s) - 1
    for i, pair in enumerate(zip(cur_s, cur_labels)):
        word = pair[0]
        if word not in vocab:
            word = '<unk>'
        label = pair[1]
        if label not in tag_list:
            tag_list.append(label)
        transition_data_count[(pre, label)] += 1
        state_count[pre] += 1
        emission_data_count[(label, word)] += 1
        pre = label

tran_dict = defaultdict(float)
emission_dict = defaultdict(float)

for key, val in transition_data_count.items():
    state = key[0]
    next_state = key[1]
    tran_dict[str(key)] = val / state_count[state]

for key, val in emission_data_count.items():
    state = key[0]
    word = key[1]
    emission_dict[str(key)] = val / state_count[state]

print("Transition parameter size: ", len(tran_dict))
print("Emission parameter size: ", len(emission_dict))
hmm_file = {' transition': tran_dict, 'emission': emission_dict}
with open("hmm.json", "w") as f_out:
    json.dump(hmm_file, f_out, indent=4)


Transition parameter size:  1392
Emission parameter size:  30303


Q: How many transition and emission parameters in your HMM? </br>
Ans: 1392 for transition and 30303 for emission.

### Task 3: Greedy Decoding with HMM (30 points)
Using Greedy Decoding algorithm to implement HMM, and output results

In [5]:
#Task 3: Greedy Decoding with HMM
def greedy_decoding(sentence):
    tag_predictions = []
    pre = '<s>'
    for w in sentence:
        if w not in vocab:
            w = '<unk>'
        cur_tag = None
        max_prob = float('-inf')
        for tag in tag_list:
            cur_prob = tran_dict[str((pre, tag))] * emission_dict[str((tag, w))]
            if cur_prob > max_prob:
                max_prob = cur_prob
                cur_tag = tag
        pre = cur_tag
        tag_predictions.append(cur_tag)
    return tag_predictions

with open('./data/test.json') as f:
    test_data = json.load(f)

test_result = []
for i, row in enumerate(test_data):
    cur_sentence = row['sentence']
    tag_predictions = greedy_decoding(cur_sentence)
    test_result.append({'index': i, 'sentence': cur_sentence, 'labels': tag_predictions})
    
with open("greedy.json", "w") as f_out:
    json.dump(test_result, f_out, indent=4)

In [6]:
def acc_result(predictions, ground_truth):
    total = 0
    match = 0
    for pred, truth in zip(predictions, ground_truth):
        for a, b in zip(pred, truth):
            total += 1
            if a == b:
                match += 1
    return float(match) / float(total)

with open('./data/dev.json') as f:
    dev_data = json.load(f)

predictions = []
ground_truth = []
for row in dev_data:
    cur_sentence = row['sentence']
    cur_labels = row['labels']
    ground_truth.append(cur_labels)
    predictions.append(greedy_decoding(cur_sentence))

print("Greedy Decoding with HMM's accuracy on the dev data: ", acc_result(predictions, ground_truth))

Greedy Decoding with HMM's accuracy on the dev data:  0.9350221601602817


Q: What is the accuracy on the dev data? </br>
Ans: 93.5%

### Task 4: Viterbi Decoding with HMM (30 Points)
Using Viterbi Decoding algorithm to implement HMM, and output results

In [7]:
# Viterbi Decoding with HMM
def viterbi_decoding(sentence):
    tag_predictions = []
    n = len(sentence)
    m = len(tag_list)
    v_table = [[0 for _ in range(m)] for __ in range(n)]
    from_table = [[-1 for _ in range(m)] for __ in range(n)]
    for i, w in enumerate(sentence):
        if w not in vocab:
            w = '<unk>'
        if i == 0:
            pre = '<s>'
            for j, tag in enumerate(tag_list):
                v_table[i][j] = tran_dict[str((pre, tag))] * emission_dict[str((tag, w))]
        else:
            for j, tag in enumerate(tag_list):
                for k, tag_pre in enumerate(tag_list):
                    cur_prob = v_table[i - 1][k] * tran_dict[str((tag_pre, tag))] * emission_dict[str((tag, w))]
                    if cur_prob > v_table[i][j]:
                        v_table[i][j] = cur_prob
                        from_table[i][j] = k
    
    pre_tag_index = -1
    cur_tag = None
    last_prob = float('-inf')
    for tag_index in range(m):
        if v_table[n-1][tag_index] > last_prob:
            last_prob = v_table[-1][tag_index]
            cur_tag = tag_list[tag_index]
            pre_tag_index = from_table[-1][tag_index]
    tag_predictions.append(cur_tag)
    
    for i in range(n - 2, -1, -1):
        cur_tag = tag_list[pre_tag_index]
        pre_tag_index = from_table[i][pre_tag_index]
        tag_predictions.append(cur_tag)
    tag_predictions.reverse()
    
    return tag_predictions

test_result = []
for i, row in enumerate(test_data):
    cur_sentence = row['sentence']
    tag_predictions = viterbi_decoding(cur_sentence)
    test_result.append({'index': i, 'sentence': cur_sentence, 'labels': tag_predictions})
    
with open("viterbi.json", "w") as f_out:
    json.dump(test_result, f_out, indent=4)

In [8]:
predictions = []
ground_truth = []
for row in dev_data:
    cur_sentence = row['sentence']
    cur_labels = row['labels']
    ground_truth.append(cur_labels)
    predictions.append(viterbi_decoding(cur_sentence))

print("Viterbi Decoding with HMM's accuracy on the dev data: ", acc_result(predictions, ground_truth))

Viterbi Decoding with HMM's accuracy on the dev data:  0.9473013174670634


Q: What is the accuracy on the dev data? </br>
Ans: 94.7%